In [1]:
from nfl import process #saved as a package the make_feature function defined in previous notebook
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer, RobustScaler, StandardScaler

from sklearn.linear_model import LinearRegression, ElasticNetCV

from sklearn.grid_search import GridSearchCV

from time import ctime

from sklearn.cross_validation import cross_val_score

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split, ShuffleSplit
from sklearn.ensemble import ExtraTreesRegressor

import statsmodels.formula.api as smf
from statsmodels.api import add_constant

from sklearn.metrics import mean_squared_error, r2_score 

from sklearn.svm import SVR, LinearSVR

import scipy.stats as stats
import math

import time

from sklearn.linear_model import Lasso, Ridge, RidgeCV


%matplotlib inline

### The general game plan is as follows. Start with several years of data for each position, and generate week by week data for the past by re-training the model on all of the prior data.

In [2]:
df = pd.read_csv("~/ProjectData/NFL/all_data_10_13_v2.csv", low_memory=False)

In [3]:
wr = process.make_feature("WR", 2010, df = df)
wr.head(3)

/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_20.0,dummy_21.0,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan
0,2963.0,TS-2300,3.0,42.2,Torrey,Smith,2011.0,-0.500000,0.045455,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2984.0,TS-2300,4.0,1.1,Torrey,Smith,2011.0,-0.416667,0.090909,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3007.0,TS-2300,6.0,11.4,Torrey,Smith,2011.0,0.083333,0.727273,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [4]:
qb = process.make_feature("QB", 2010, df = df)
qb.head(3)

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_20.0,dummy_21.0,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan
0,2948.0,AS-1600,2.0,15.26,Alex,Smith,2011.0,-0.333333,-0.470588,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2961.0,AS-1600,3.0,8.04,Alex,Smith,2011.0,-0.666667,-0.611765,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2976.0,AS-1600,4.0,18.54,Alex,Smith,2011.0,-0.083333,-0.894118,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
te = process.make_feature("TE", 2010, df = df)
te.head(3)

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_20.0,dummy_21.0,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan
0,2970.0,MB-1800,4.0,4.2,Martellus,Bennett,2011.0,0.250000,0.363636,0.666667,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3009.0,MB-1800,6.0,3.3,Martellus,Bennett,2011.0,1.833333,0.409091,0.666667,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3022.0,MB-1800,7.0,0.0,Martellus,Bennett,2011.0,-0.250000,1.045455,0.666667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
rb = process.make_feature("RB", 2010, df = df)
rb.head(3)

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_20.0,dummy_21.0,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan
0,2977.0,BM-2400,4.0,1.3,Brit,Miller,2011.0,-0.166667,-0.318182,0.333333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3048.0,BM-2400,9.0,0.1,Brit,Miller,2011.0,-0.583333,-0.590909,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3059.0,BM-2400,10.0,3.2,Brit,Miller,2011.0,-1.250000,-0.954545,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
wr_X, wr_y, wr_frame = process.crop_and_separate(0, 1, wr, "data")

/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [8]:
qb_X, qb_y , qb_frame = process.crop_and_separate(0, 1, qb, "data")

In [9]:
rb_X, rb_y, rb_frame = process.crop_and_separate(0, 1, rb, "data")

In [10]:
te_X, te_y, te_frame = process.crop_and_separate(0, 1, te, "data")

###### PROTOTYPE THE PREDICTION LOOP

In [11]:
#the loop will use the frame to figure out the X weights and the indexes on which to fit/predict the X and y
X, y, frame = process.crop_and_separate(0, 2, wr, "data")
frame.head()

,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_20.0,dummy_21.0,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan
0,2963.0,TS-2300,3,42.2,Torrey,Smith,2011,-0.500000,0.045455,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2984.0,TS-2300,4,1.1,Torrey,Smith,2011,-0.416667,0.090909,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3007.0,TS-2300,6,11.4,Torrey,Smith,2011,0.083333,0.727273,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3024.0,TS-2300,7,6.2,Torrey,Smith,2011,-0.833333,0.272727,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,3027.0,TS-2300,8,8.7,Torrey,Smith,2011,-0.250000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
#define model, using the RF with the attributes that I grid searched for in notebook 3
et4 = ExtraTreesRegressor(max_depth=7,
                          min_samples_leaf= 30, 
                          min_weight_fraction_leaf=0.13740419750125152, 
                          n_estimators=50, n_jobs = -1)



In [13]:
for year in set(frame.seas): print year

2011
2012
2013
2014
2015


In [14]:
sy = 2014 #years to predict forward from
cy = 2015 #looping current year of prediction
cw = 2 # looping current week of prediction
player = "TS-2300"

In [15]:
for week in set(frame[frame.seas == sy].wk): print week

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [16]:
#define prediction index
predict_rows = frame[(frame.seas == cy) & (frame.wk == cw) & (frame.player == player)].index

In [17]:
#define training index
train_rows = frame[(((frame.seas == cy) & (frame.wk <=cw)) | (frame.seas < cy)) ].index

#### Another thing to do here is to, compute a sample weighting vector, but i want to time it as well to see if its realistic to re-fit per player per week. The weighting would apply a multiplication factor to the players score and weight by ratio of scores so it would be two fold, favor the higher perfomances since that is what I am targeting and also add an extra weight to each particular 

In [18]:
frame["weight"] = frame.target

In [19]:
%%timeit

#a = time.time()


frame.ix[frame.player == player,"weight"] = frame.ix[frame.player == player,"weight"] *10 
frame.weight = frame.weight/sum(frame.weight)
#print sum(frame.weight)

et4.fit(X.ix[train_rows], y.ix[train_rows], sample_weight= np.array(frame.weight.ix[train_rows]))
x = et4.predict(X.ix[predict_rows])
#b = time.time()

#print b-a

1 loop, best of 3: 2.28 s per loop


##### PREDICTION FUNCTION

In [20]:
def generate_prediction(X, y, frame, sy, model, w=10):
    """generates predictions by player by week with the option to increase the weight of the players
    historic performances. w = 1 would weight every players game by the amount of fp scored without
    increasing the weight of each player's own games. w=0 does not apply any sample weight.
    
    frame = dataframe resulting from the make feature function
    sy = starting year to predict from
    w = factor to apply to each players scores when weighting samples
    model = predictor with a fit/predict method"""
    yr_set = pd.Series([_ for _ in set(frame.seas)])
    cycle_yrs = yr_set[yr_set>=sy]
    for year in cycle_yrs:
        week_set = pd.Series([_ for _ in set(frame.wk[frame.seas==year])])
        for week in week_set:
            player_set = pd.Series([_ for _ in set(frame.player[(frame.seas == year) & (frame.wk == week)])])
            for player in player_set:
                predict_rows = frame[(frame.seas == year) & (frame.wk == week) & (frame.player == player)].index
                train_rows = frame[(((frame.seas == year) & (frame.wk <=week)) | (frame.seas < year)) ].index
                
                if w == 0:
                    model.fit(X.ix[train_rows], y.ix[train_rows])
                else:
                    print player, year, week
                    frame["weight"] = frame.target
                    frame.ix[frame.player == player,"weight"] = frame.ix[frame.player == player,"weight"] *w
                    frame.weight = frame.weight/sum(frame.weight)
                    model.fit(X.ix[train_rows], y.ix[train_rows], sample_weight= np.array(frame.weight.ix[train_rows]))
                
                frame.ix[predict_rows,"prediction"] = model.predict(X.ix[predict_rows])
                
            print "Week ", week, " year ", year, " is done."
        
    return frame
    
    

In [21]:

lr = Lasso()

In [22]:
a= time.time()
qb_result = generate_prediction(qb_X, qb_y, qb_frame, 2014, lr, w=0)
print time.time() - a

Week  1  year  2014  is done.
Week  2  year  2014  is done.
Week  3  year  2014  is done.
Week  4  year  2014  is done.
Week  5  year  2014  is done.
Week  6  year  2014  is done.
Week  7  year  2014  is done.
Week  8  year  2014  is done.
Week  9  year  2014  is done.
Week  10  year  2014  is done.
Week  11  year  2014  is done.
Week  12  year  2014  is done.
Week  13  year  2014  is done.
Week  14  year  2014  is done.
Week  15  year  2014  is done.
Week  16  year  2014  is done.
Week  17  year  2014  is done.
Week  18  year  2014  is done.
Week  19  year  2014  is done.
Week  20  year  2014  is done.
Week  21  year  2014  is done.
Week  1  year  2015  is done.
Week  2  year  2015  is done.
Week  3  year  2015  is done.
Week  4  year  2015  is done.
Week  5  year  2015  is done.
Week  6  year  2015  is done.
Week  7  year  2015  is done.
Week  8  year  2015  is done.
Week  9  year  2015  is done.
Week  10  year  2015  is done.
Week  11  year  2015  is done.
Week  12  year  2015  is d

In [23]:
a= time.time()
wr_result = generate_prediction(wr_X, wr_y, wr_frame, 2014, lr, w=0)
print time.time() - a

Week  1  year  2014  is done.
Week  2  year  2014  is done.
Week  3  year  2014  is done.
Week  4  year  2014  is done.
Week  5  year  2014  is done.
Week  6  year  2014  is done.
Week  7  year  2014  is done.
Week  8  year  2014  is done.
Week  9  year  2014  is done.
Week  10  year  2014  is done.
Week  11  year  2014  is done.
Week  12  year  2014  is done.
Week  13  year  2014  is done.
Week  14  year  2014  is done.
Week  15  year  2014  is done.
Week  16  year  2014  is done.
Week  17  year  2014  is done.
Week  18  year  2014  is done.
Week  19  year  2014  is done.
Week  20  year  2014  is done.
Week  21  year  2014  is done.
Week  1  year  2015  is done.
Week  2  year  2015  is done.
Week  3  year  2015  is done.
Week  4  year  2015  is done.
Week  5  year  2015  is done.
Week  6  year  2015  is done.
Week  7  year  2015  is done.
Week  8  year  2015  is done.
Week  9  year  2015  is done.
Week  10  year  2015  is done.
Week  11  year  2015  is done.
Week  12  year  2015  is d

In [24]:
a= time.time()
rb_result = generate_prediction(rb_X, rb_y, rb_frame, 2014, lr, w=0)
print time.time() - a

Week  1  year  2014  is done.
Week  2  year  2014  is done.
Week  3  year  2014  is done.
Week  4  year  2014  is done.
Week  5  year  2014  is done.
Week  6  year  2014  is done.
Week  7  year  2014  is done.
Week  8  year  2014  is done.
Week  9  year  2014  is done.
Week  10  year  2014  is done.
Week  11  year  2014  is done.
Week  12  year  2014  is done.
Week  13  year  2014  is done.
Week  14  year  2014  is done.
Week  15  year  2014  is done.
Week  16  year  2014  is done.
Week  17  year  2014  is done.
Week  18  year  2014  is done.
Week  19  year  2014  is done.
Week  20  year  2014  is done.
Week  21  year  2014  is done.
Week  1  year  2015  is done.
Week  2  year  2015  is done.
Week  3  year  2015  is done.
Week  4  year  2015  is done.
Week  5  year  2015  is done.
Week  6  year  2015  is done.
Week  7  year  2015  is done.
Week  8  year  2015  is done.
Week  9  year  2015  is done.
Week  10  year  2015  is done.
Week  11  year  2015  is done.
Week  12  year  2015  is d

In [25]:
a= time.time()
te_result = generate_prediction(te_X, te_y, te_frame, 2014, lr, w=0)
print time.time() - a

Week  1  year  2014  is done.
Week  2  year  2014  is done.
Week  3  year  2014  is done.
Week  4  year  2014  is done.
Week  5  year  2014  is done.
Week  6  year  2014  is done.
Week  7  year  2014  is done.
Week  8  year  2014  is done.
Week  9  year  2014  is done.
Week  10  year  2014  is done.
Week  11  year  2014  is done.
Week  12  year  2014  is done.
Week  13  year  2014  is done.
Week  14  year  2014  is done.
Week  15  year  2014  is done.
Week  16  year  2014  is done.
Week  17  year  2014  is done.
Week  18  year  2014  is done.
Week  19  year  2014  is done.
Week  20  year  2014  is done.
Week  21  year  2014  is done.
Week  1  year  2015  is done.
Week  2  year  2015  is done.
Week  3  year  2015  is done.
Week  4  year  2015  is done.
Week  5  year  2015  is done.
Week  6  year  2015  is done.
Week  7  year  2015  is done.
Week  8  year  2015  is done.
Week  9  year  2015  is done.
Week  10  year  2015  is done.
Week  11  year  2015  is done.
Week  12  year  2015  is d

In [26]:
#save the prediction frames down so i can run some analysis in the next notebook

In [27]:
qb_result["pos"] = "QB"
wr_result["pos"] = "WR"
rb_result["pos"] = "RB"
te_result["pos"] = "TE"

In [28]:
grab_cols = ["target", "prediction", "gid", "player",  "pos", "fname", "lname", "seas", "wk"]

qb_result.head(1)



,gid,player,wk,target,fname,lname,seas,ou,imp_score,height,...,dummy_3.0,dummy_4.0,dummy_5.0,dummy_6.0,dummy_7.0,dummy_8.0,dummy_9.0,dummy_nan,prediction,pos
0,2948.0,AS-1600,2,15.26,Alex,Smith,2011,-0.333333,-0.470588,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,QB


In [29]:
final = qb_result[grab_cols].append(wr_result[grab_cols]).append(rb_result[grab_cols]).append(te_result[grab_cols])

In [33]:
final.to_csv("predictions_10_20.csv")

In [31]:
test = final[["target", "prediction"]].dropna()

In [32]:
r2_score(test.target, test.prediction)

0.33249462172245048

#### The results of the RF are not here but I was not getting a good R2 score (close to 0) which may have been due to overfitting in the third notebook but it looks like a Lasso model works failry well here across all positions so I will move on to the next step which is the line up optimization.